In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk import FreqDist

import string

from proyecto_ampliacion import Proyecto_ampliacion
from funciones_extra import *

In [ ]:
ej1 = "El proyecto consiste en el aumento de capacidad de la subestación Cañete mediante la instalación de un nuevo transformador 66/23 kV y al menos 16 MVA de capacidad con Cambiador de Derivación Bajo Carga (CDBC), y sus respectivos paños de conexión en ambos niveles de tensión. A su vez, el proyecto incluye la ampliación de la barra e instalaciones comunes del patio de 66 kV de la subestación, cuya configuración corresponde a barra simple, la cual deberá considerar espacio en barra y plataforma para dos posiciones, de manera de permitir la conexión del nuevo transformador de poder antes mencionado y la normalización de la conexión de la línea 1x66 kV Tres Pinos – Cañete. Adicionalmente, el proyecto considera la construcción de una nueva sección de barra de 23 kV, en configuración barra simple, contemplándose la construcción de, al menos, dos paños para alimentadores, el paño de conexión del transformador antes mencionado, un paño para la conexión de un banco de condensadores y la construcción de un paño de interconexión con la barra existente. En caso de definirse el desarrollo de la ampliación de este patio como una sala de celdas, se deberán considerar los paños contenidos en esta descripción junto con la construcción de una celda para equipos de medida, la construcción de una celda para servicios auxiliares si corresponde. Además, el proyecto contempla la instalación de dos nuevos bancos de condensadores de 3,6 MVAr en 23 kV, de dos etapas cada uno, conectados a cada sección de barra de 23 kV con sus respectivos paños de conexión."


In [ ]:

def separar_por_patios(texto):
    parrafos = texto.split(".")
    palabras_clave_patios = ["patio"]
    palabras_clave_celdas = ["sala de celdas"]
    palabras_clave_paños = ["paño", "paños"]
    palabras_posiciones = ["diagonales", "posiciones", "posición", "diagonal"]
    parrafos_patios = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_patios) and "configuración" in parrafo and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_celdas = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_celdas) and "configuración" in parrafo and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_paños = [parrafo for parrafo in parrafos if any(palabra in parrafo for palabra in palabras_clave_paños) and "configuración" and any(palabra in parrafo for palabra in palabras_posiciones)]
    parrafos_otros = [parrafo for parrafo in parrafos if parrafo not in parrafos_patios and parrafo not in parrafos_celdas and parrafo not in parrafos_paños]
    
    diccionario_parrafos = {
        "patios": parrafos_patios,
        "celdas": parrafos_celdas,
        "paños": parrafos_paños,
        "otros": parrafos_otros
    }

    return diccionario_parrafos
    

In [ ]:
def remove_stopwords(texto):
    stop_words = set(stopwords.words('spanish'))
    tokens = nltk.word_tokenize(texto)
    tokens = [word for word in tokens if word.lower() not in stop_words]
    tokens = [word for word in tokens if word.lower() not in string.punctuation]
    return " ".join(tokens)

In [ ]:
def clasificar_parrafo(parrafo):
    tipo_aumento_capacidad = ["instalación nuevo transformador", "reemplazo actual transformador"] #caso inst o const trafo
    tipo_ampliacion_construccion = ["ampliación barra", "construcción nueva sección barra", "ampliación galpón", "ampliación patio", "construcción nueva barra", "ampliación sala celdas", "construcción nueva sala celdas", "construcción nuevo paño"] # caso ampliacion construccion patio, paño, nva barra, etc
    tipo_otro = ["nuevos bancos condensadores", "nuevo banco condensadores", "banco autotransformadores existente nueva"]

    tipo_no_interesa = ["proyecto incluye todas obras modificaciones", "respectivas bases licitación", "contempla todas tareas labores obras", "caso definirse desarrollo", " producto del aumento de capacidad antes", "Entrada operación", "Valor inversión", "C.O.M.A", "referenciales V.I", "referencial proyecto"]

    tipo = "REVISAR!"
    elemento_encontrado = None

    #quizas, podria devolver directamente el elemento_encontrado y definir las listas en la otra funcion, tal que si elemento encontrado esta en la lista "tipo_aumento_capacidad", se utiliza el esquema de estilo xml correspondiente al aumento de capacidad de la s/e, y asi con los otros tipos de parrafos.
    for elemento in tipo_no_interesa:
        if elemento in parrafo:
            tipo = "no_interesa"
            elemento_encontrado = elemento
            return tipo, None

    for elemento in tipo_aumento_capacidad:
        if elemento in parrafo:
            tipo = "construccion_instalacion_trafo"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    
    for elemento in tipo_ampliacion_construccion:
        if elemento in parrafo:
            tipo = "ampliacion_construccion_patio"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    
    for elemento in tipo_otro:
        if elemento in parrafo:
            tipo = "otro"
            elemento_encontrado = elemento
            return tipo, elemento_encontrado
    


    return tipo, elemento_encontrado
    
    


In [ ]:
parrafos = sent_tokenize(ej1)


for parrafo in parrafos:
    parrafo_limpio = remove_stopwords(parrafo)
    tipo = clasificar_parrafo(parrafo_limpio)
    print(f"Parrafo tipo {tipo}: {parrafo}")
    print("\n")


In [ ]:
def extraer_tension_trafo_reemplazado(parrafo_limpio):
    #patron = r"\d{2,3}/\d{2,3}(?:,\d)?\s*kV"
    patron = r"transformador N°\s?\d+\s+\d{1,3}/\d{1,3}(?:,\d)?\s*kV \d{1,3}(?:,\d)? MVA"

    match = re.search(patron, parrafo_limpio)

    if match:
        tension_trafo_reemplazado = match.group()
        patron_trafo_nuevo = r"transformación \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"


    else:
        tension_trafo_reemplazado = None
        patron_trafo_nuevo = r"nuevo transformador \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"


    if tension_trafo_reemplazado:
        patron_trafo_nuevo = r"transformación \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"

    else:
        patron_trafo_nuevo = r"nuevo transformador \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"

    match = re.search(patron_trafo_nuevo, parrafo_limpio)

    if match:
        tension_trafo_nuevo = match.group()

    else:
        tension_trafo_nuevo = None

    return tension_trafo_reemplazado, tension_trafo_nuevo
 


In [ ]:

def extraer_tension_trafo_reemplazado(parrafo_limpio):
    patron_reemplazo = r"transformador N°\s?\d+\s+\d{1,3}/\d{1,3}(?:,\d)?\s*kV \d{1,3}(?:,\d)? MVA"
    patron_transformacion = r"transformación \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"
    patron_nuevo = r"nuevo transformador \d{1,3}/\d{1,3}(?:,\d)?\s*kV menos \d{1,3} MVA"

    match_reemplazo = re.search(patron_reemplazo, parrafo_limpio)
    tension_trafo_reemplazado = match_reemplazo.group() if match_reemplazo else None

    # Determinar cuál patrón usar en función de si se encontró reemplazo o no
    patron_trafo_nuevo = patron_transformacion if tension_trafo_reemplazado else patron_nuevo

    match_nuevo = re.search(patron_trafo_nuevo, parrafo_limpio)
    tension_trafo_nuevo = match_nuevo.group() if match_nuevo else None

    return tension_trafo_reemplazado, tension_trafo_nuevo


In [ ]:
file = "plan_expansion_final_2023.pdf"
dic_amp = generar_diccionario_ampliaciones(file)
dic_desc_amp = generar_diccionario_descripciones_amp(file, dic_amp)


for titulo, descripcion in dic_desc_amp.items():
    
    parrafos = sent_tokenize(descripcion)
    for parrafo in parrafos:
        parrafo_limpio = remove_stopwords(parrafo)
        tipo, elemento_encontrado = clasificar_parrafo(parrafo_limpio)

        #vamos a analizar cada tipo de parrafo, para saber como proceder en cada caso:
        if tipo == "construccion_instalacion_trafo":
            print(titulo, "\n")
            print(f"Parrafo tipo {tipo}, elemento encontrado: {elemento_encontrado}: \n")
            print(parrafo_limpio)
            tension_trafo_reemplazado, tension_trafo_nuevo = extraer_tension_trafo_reemplazado(parrafo_limpio)
            print(f"Tension trafo reemplazado: {tension_trafo_reemplazado}")
            print(f"Tension trafo nuevo: {tension_trafo_nuevo}")

    print("\n")

In [125]:
def extraer_tension(parrafo):
    pattern = re.compile(r'(patio|sala celdas|nueva barra|sección barra|nuevo paño|ampliación barra|ampliación barras) \d+(?:,\d+)? kV', re.IGNORECASE)
    match = pattern.search(parrafo)

    if match:
        return match.group(0)
    
    else:
        return "Buscar en Informe"



def extraer_configuracion(parrafo):
    l_config_oficial = ['barra principal seccionada y barra de transferencia', 'interruptor y medio', 'doble barra principal y barra de transferencia', 'doble barra principal con barra de transferencia', 'barra simple', 'barra principal con barra de transferencia', 'barra principal más barra auxiliar', 'barra simple seccionada', 'barra principal y barra de transferencia']

    for config in l_config_oficial:
        if config in parrafo:
            return config
        
    return "No se pudo extraer la configuración"



def extraer_numero_posiciones_v3(parrafo):
    numeros = {
        "uno": 1,
        "una": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9,
        "diez": 10
    }
    pattern1 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez)(?: (nueva|nuevas))? (posición|posiciones|diagonal|diagonales)\b', re.IGNORECASE)
    pattern2 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (celda para alimentador|celdas para alimentadores)\b', re.IGNORECASE)
    pattern3 = re.compile(r'\b(\d+|uno|una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez) (paño|paños)\b', re.IGNORECASE)


    match1 = pattern1.search(parrafo)
    match2 = pattern2.search(parrafo)
    match3 = pattern3.search(parrafo)

    if match1:
        numero_str, _, tipo = match1.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)

        if tipo.lower() == "diagonales" or tipo.lower() == "diagonal":
            return numero * 2
        elif tipo.lower() == "posiciones" or tipo.lower() == "posición":
            return numero
    
    elif match2:
        numero_str, tipo = match2.groups()
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} celda(s) para alimentadores"

    elif match3:
        numero_str = match3.group(1)
        numero = numeros.get(numero_str.lower(), int(numero_str) if numero_str.isdigit() else 0)
        return f"al menos {numero} paño(s) futuro(s)"

    else:
        return "Buscar en Informe"


def extraer_conexiones(parrafo):

    def limpiar_conexion(conexion):
    # Eliminar artículos y preposiciones iniciales
        conexion = re.sub(r'^(el |la |los |las |del |de |a |en |para |la conexión de |la construcción de |del |un |una |)', '', conexion, flags=re.IGNORECASE).strip()
        return conexion
    
    try:
        patron_conexiones_inicio = re.compile(r'\bde manera de permitir la conexión\b(.*?)(\.\s|$)', re.IGNORECASE | re.DOTALL)
        patron_alternativo = re.compile(r'\bpaños para (alimentador|alimentadores)\b(.*?)(\.\s|$)', re.IGNORECASE | re.DOTALL)


        match_conexiones_inicio = patron_conexiones_inicio.search(parrafo)
        match_alternativo = patron_alternativo.search(parrafo)

        if match_conexiones_inicio:
            
            conexiones = match_conexiones_inicio.group(1).replace("y la", ",").split(",")
            lista_conexiones = [limpiar_conexion(conexion.strip()) for conexion in conexiones]
            return lista_conexiones

        elif match_alternativo:
            conexiones = match_alternativo.group(0).split(",")
            lista_conexiones = [limpiar_conexion(conexion.strip()) for conexion in conexiones]
            return lista_conexiones




    except Exception as e:
        print(f"Error extraccion conexiones: {e}")


    return "Revisar manualmente, al parecer no hay conexiones en el texto."


def calcular_posiciones_disponibles(lista_conexiones, posiciones):
    numeros = {
        "uno": 1,
        "una": 1,
        "dos": 2,
        "tres": 3,
        "cuatro": 4,
        "cinco": 5,
        "seis": 6,
        "siete": 7,
        "ocho": 8,
        "nueve": 9,
        "diez": 10
    }

    posiciones_disponibles = posiciones

    if isinstance(posiciones, str):
        return posiciones_disponibles
    
    else:
        for conexion in lista_conexiones:
            if "seccionamiento" in conexion:
                patron = r'\b(\d+)x\d{2,3}\b'
                match = re.findall(patron, conexion)
                if match:
                    for tension in match:
                        posic_ocup = int(tension[0])*2 #OJO ACA, pq [0]? 
                        posiciones_disponibles -= posic_ocup
                else:
                    print("No se encontraron coincidencias")

            elif ("la conexión de la línea") in conexion:
                patron = r'\b(\d+)x\d{2,3}\b'
                match = re.search(patron, conexion)
                if match:
                    posic_ocup = int(match.group()[0]) #Ojo aca tmb
                    posiciones_disponibles -= posic_ocup
                else:
                    print("No se encontraron coincidencias")

            elif "la conexión de la nueva línea" in conexion:
                patron = r'\b(\d+)x\d{2,3}\b'
                match = re.search(patron, conexion)
                if match:
                    posic_ocup = int(match.group()[0])
                    posiciones_disponibles -= posic_ocup
                else:
                    print("No se encontraron coincidencias")


            elif "la conexión de las líneas" in conexion:
                patron = r'\b(\d+)x\d{2,3}\b'
                match = re.search(patron, conexion)

                if match:
                    posic_ocup = int(match.group()[0])
                    posiciones_disponibles -= posic_ocup
                else:
                    print("No se encontraron coincidencias")

            elif "la conexión de la obra" in conexion:
                patron = r'\b(\d+)x\d{2,3}\b'
                coincidencias = re.findall(patron, conexion)

                if coincidencias:
                    for coincidencia in coincidencias:
                        posic_ocup = int(coincidencia[0])
                        posiciones_disponibles -= posic_ocup

                else:
                    print("Problemas posiciones nuevas obras, calculo posiciones.")


            elif "transformador" in conexion and "banco" not in conexion:
                posiciones_disponibles -= 1

            elif "paño" in conexion:
                if "paño acoplador" in conexion and "paño seccionador" in conexion:
                    posiciones_disponibles -= 2

                elif "paño acoplador" in conexion:
                    posiciones_disponibles -= 1

                elif "paño seccionador" in conexion:
                    posiciones_disponibles -= 1

                elif "un paño para la línea" in conexion:
                    patron = r'\b(\d+)x\d{2,3}\b'
                    coincidencias = re.findall(patron, conexion)

                    if coincidencias:
                        for coincidencia in coincidencias:
                            posic_ocup = int(coincidencia[0])
                            posiciones_disponibles -= posic_ocup

                    else:
                        continue

                else:
                    print("Revisar tipo de paño a conectar xdddd")
                    continue

            else:
                patron_tension = r'\b(\d+)x\d{2,3}\b'
                match = re.findall(patron_tension, conexion)
                if match:
                    for tension in match:
                        posic_ocup = int(tension)
                        posiciones_disponibles -= posic_ocup

    return posiciones_disponibles



In [126]:
file = "plan_expansion_final_2023.pdf"
dic_amp = generar_diccionario_ampliaciones(file)
dic_desc_amp = generar_diccionario_descripciones_amp(file, dic_amp)


for titulo, descripcion in dic_desc_amp.items():
    
    parrafos = sent_tokenize(descripcion)
    for parrafo in parrafos:
        parrafo_limpio = remove_stopwords(parrafo)
        tipo, elemento_encontrado = clasificar_parrafo(parrafo_limpio)

        #vamos a analizar cada tipo de parrafo, para saber como proceder en cada caso:
        if tipo == "ampliacion_construccion_patio":
            print(titulo)
            print(f"Parrafo tipo {tipo}, elemento encontrado: {elemento_encontrado}")
            print(parrafo_limpio)
            tension = extraer_tension(parrafo_limpio)
            print(f"Tensión: {tension}")
            configuracion = extraer_configuracion(parrafo)
            print(f"Configuración: {configuracion}")
            numero_posiciones = extraer_numero_posiciones_v3(parrafo)
            print(f"Número de posiciones: {numero_posiciones}")
            conexiones = extraer_conexiones(parrafo)
            print(f"Conexiones: {conexiones}")
            posiciones_disponibles = calcular_posiciones_disponibles(conexiones, numero_posiciones)
            print(f"Posiciones disponibles: {posiciones_disponibles}")
            print("\n")

Ampliación en S/E Cóndores 110 kV (BP+BT) 
Parrafo tipo ampliacion_construccion_patio, elemento encontrado: ampliación barra
Descripción general ubicación obra proyecto consiste ampliación barras instalaciones comunes patio 110 kV subestación Cóndores cuya configuración corresponde barra principal barra transferencia dos nuevas posiciones manera permitir conexión obra “ Nueva S/E Alto Molle nueva línea 2x110 kV Alto Molle – Cóndores ”
Tensión: patio 110 kV
Configuración: barra principal con barra de transferencia
Número de posiciones: 2
Conexiones: ['la obra “Nueva S/E Alto Molle y nueva línea 2x110 kV Alto Molle – Cóndores”.']
Posiciones disponibles: 0


Ampliación en S/E Nueva Cardones 220 kV (IM) 
Parrafo tipo ampliacion_construccion_patio, elemento encontrado: ampliación galpón
Descripción general ubicación obra proyecto consiste ampliación galpón instalaciones comunes patio 220 kV subestación Nueva Cardones cuya configuración corresponde interruptor medio cuatro nuevas diagonales 